# Monte Carlo Simulation - Project

### ***What is the Monte Carlo Simulation?***

A Monte Carlo simulation is used to model the probability of different outcomes in a process that cannot easily be predicted due to the intervention of random variables. It is a technique used to understand the impact of risk and uncertainty.

A Monte Carlo simulation is used to tackle a range of problems in many fields including investing, business, physics, and engineering.

It is also referred to as a multiple probability simulation.


Source: https://www.investopedia.com/terms/m/montecarlosimulation.asp#:~:text=Monte%20Carlo%20Simulation%20Steps&text=There%20are%20two%20components%20to,input%2C%20which%20represents%20market%20volatility.

In [1]:
!pip install -q yfinance


import yfinance as yf

import pandas as pd
import numpy as np
import plotly.express as px
import seaborn as sns
from scipy import stats

     |████████████████████████████████| 62 kB 519 kB/s 


In [2]:
#Sets the stock that we're gonna use to make the Monte Carlo Simulation

ac = ['BOVA11.SA']
ac

['BOVA11.SA']

In [5]:
#Downloads the historical price
ac_df = pd.DataFrame(yf.download(ac, '2015-01-01')['Close'])
ac_df.to_csv('stock.csv')

ac_df = pd.read_csv('stock.csv')
ac_df

[*********************100%***********************]  1 of 1 completed


,Date,Close
0,2015-01-02,47.259998
1,2015-01-05,46.320000
2,2015-01-06,46.580002
3,2015-01-07,48.150002
4,2015-01-08,48.509998
...,...,...
1887,2022-08-05,102.739998
1888,2022-08-08,104.800003
1889,2022-08-09,105.050003
1890,2022-08-10,106.559998


In [9]:
#Drops the 'Date' column
ac_df.drop(columns = 'Date', inplace=True)
ac_df

,Close
0,47.259998
1,46.320000
2,46.580002
3,48.150002
4,48.509998
...,...
1887,102.739998
1888,104.800003
1889,105.050003
1890,106.559998


In [11]:
#Normalizes the return of the stock
ac_df_norm = ac_df.copy()

for i in ac_df_norm.columns:
  ac_df_norm[i] = ac_df_norm[i] / ac_df_norm[i][0]

ac_df_norm

,Close
0,1.000000
1,0.980110
2,0.985612
3,1.018832
4,1.026449
...,...
1887,2.173931
1888,2.217520
1889,2.222810
1890,2.254761


In [13]:
#Calculates the daily returns of the stock
daily_return = np.log(1 + ac_df_norm.pct_change())

daily_return.fillna(0, inplace=True)

daily_return

,Close
0,0.000000
1,-0.020090
2,0.005597
3,0.033150
4,0.007449
...,...
1887,0.002534
1888,0.019852
1889,0.002383
1890,0.014272


# Calculating the Drift

Drift, is constant directional movement of the simulation

In [14]:
# Calculates the average daily return
mean_returns = daily_return.mean()
mean_returns

Close    0.000426
dtype: float64

In [15]:
# Calculates Variance
var_returns = daily_return.var()
var_returns

Close    0.000273
dtype: float64

In [21]:
# Calculates the Standard deviation

std_returns = daily_return.std()
std_returns

Close    0.016509
dtype: float64

In [16]:
# Calculates the Drift

drift = mean_returns - (0.5 * var_returns)
drift

Close    0.00029
dtype: float64

# Calculating Daily Returns

In [17]:
#defines how many days ahead are going to be simulated, and how many simulations are going to be done

days_ahead = 246
simulations = 10

In [18]:
# Creates a random matrix with a normal distribution that's going to be used in the calculate the simulation

Z = stats.norm.ppf(np.random.rand(days_ahead, simulations))

In [19]:
Z.shape

(246, 10)

In [22]:
# Calculates a matrix with the 'future' daily variations, based on the past behavior of the stock

future_daily_return = np.exp(drift.values + (std_returns.values * Z))

In [23]:
future_daily_return.shape

(246, 10)

# Predicting future prices

In [24]:
# Creates a matrix filled with zeroes with the same shape as the 'future_daily_return' matrix
predictions = np.zeros_like(future_daily_return)

predictions.shape

(246, 10)

In [27]:
# Looks for the last stock price observed in the dataframe

ac_df.iloc[-1]

(Close    105.879997
 Name: 1891, dtype: float64, array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]))

In [28]:
# Looks for the first array in the predictions matrix

predictions[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [30]:
# Sets the first array in the zeroes_like matrix as the last stock price observed.

# This is the starting point of the simulation

predictions[0] = ac_df.iloc[-1]
predictions

array([[105.87999725, 105.87999725, 105.87999725, ..., 105.87999725,
        105.87999725, 105.87999725],
       [  0.        ,   0.        ,   0.        , ...,   0.        ,
          0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        , ...,   0.        ,
          0.        ,   0.        ],
       ...,
       [  0.        ,   0.        ,   0.        , ...,   0.        ,
          0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        , ...,   0.        ,
          0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        , ...,   0.        ,
          0.        ,   0.        ]])

In [32]:
# Calculates the future stock prices, based on the stock price from the day before.

# The first prediction is based on the last observed stock price

for day in range(1, days_ahead):
  predictions[day] = predictions[day - 1] * future_daily_return[day]

In [33]:
predictions.shape

(246, 10)

In [34]:
predictions

array([[105.87999725, 105.87999725, 105.87999725, ..., 105.87999725,
        105.87999725, 105.87999725],
       [107.63502181, 108.4378691 , 105.33423534, ..., 103.40912354,
        107.1523384 , 105.81254781],
       [109.40766728, 106.33775304, 102.51869747, ..., 103.57243479,
        109.43550818, 104.77520416],
       ...,
       [141.86865222, 118.77559646, 107.50534435, ..., 107.82215088,
        107.69701054, 105.44241783],
       [142.41411634, 120.00303936, 107.87868673, ..., 107.36343895,
        106.0746242 , 103.39214427],
       [138.56726872, 118.43368936, 108.99760245, ..., 105.70021721,
        104.65031567, 102.46852942]])

# Plotting

In [36]:
# Trasposes the array

predictions.T

array([[105.87999725, 107.63502181, 109.40766728, ..., 141.86865222,
        142.41411634, 138.56726872],
       [105.87999725, 108.4378691 , 106.33775304, ..., 118.77559646,
        120.00303936, 118.43368936],
       [105.87999725, 105.33423534, 102.51869747, ..., 107.50534435,
        107.87868673, 108.99760245],
       ...,
       [105.87999725, 103.40912354, 103.57243479, ..., 107.82215088,
        107.36343895, 105.70021721],
       [105.87999725, 107.1523384 , 109.43550818, ..., 107.69701054,
        106.0746242 , 104.65031567],
       [105.87999725, 105.81254781, 104.77520416, ..., 105.44241783,
        103.39214427, 102.46852942]])

In [39]:
# Plots all the simulations made

figura = px.line(title = 'Future Stock Prices - Simulations')
for i in range(len(predictions.T)):
  figura.add_scatter(y = predictions.T[i], name = f'Simulation {i}')

figura.show()

In [56]:
# Looks for the best simulation outcome

best_simulation = predictions[:][-1].max()
best_simulation

138.56726871689173

In [57]:
# Looks for the worst simulation outcome

worst_simulation = predictions[:][-1].min()
worst_simulation

86.59373926716789

In [65]:
# Looks for the best and worst simulation indexes

for i, b in enumerate(predictions[:][-1]):
  #print(i, b)
  if b == best_simulation:
    best_index = i
  elif b == worst_simulation:
    worst_index = i

print(best_index, worst_index)

0 6


In [66]:
#Plots the best and the worst simulations

figura = px.line(title = 'Future Stock Prices - Simulations')

figura.add_scatter(y = predictions.T[worst_index], name = f'Worst Simulation')
figura.add_scatter(y = predictions.T[best_index], name = f'Best Simulation')

figura.show()